In [2]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/msmarco-bert-base-dot-v5')

2022-04-25 12:12:10.307209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 12:12:10.307246: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import json

train = [json.loads(l) for l in open("../data/COPA/train.jsonl").readlines()]
val = [json.loads(l) for l in open("../data/COPA/val.jsonl").readlines()]
test = [json.loads(l) for l in open("../data/COPA/test.jsonl").readlines()]

train = train + val

In [4]:
passages = ["Context: {} Alternative 1: {} Alternative 2: {}".format(item['premise'], item['choice1'], item['choice2']) for item in train]

In [5]:
passages_embeddings = model.encode(passages, show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [8]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate(prompt, model="text-davinci-002", temperature=0.0,presence_penalty=2 ):
  response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt,
    temperature=temperature,
    max_tokens=64,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=presence_penalty,
    stop=["[END]"],
    logprobs=1
  )
  return response['choices'][0]['text'].strip()

In [ ]:
from tqdm import tqdm
import numpy as np
import json

kshot = 32
model_name = "text-davinci-002"

logs = []

results_file = open("../results/COPA_{0}_{1}shot.jsonl".format(model_name, kshot), "w")

for item in tqdm(test):
    query = "Context: {0} Alternative 1: {1} Alternative 2: {2}".format(item['premise'], item['choice1'], item['choice2'])
    query_embedding = model.encode(query)

    topk = util.dot_score(query_embedding, passages_embeddings)[0].topk(kshot)

    hits = np.array(train)[topk.indices]

    prompt = ""
    for hit in hits:
        template = "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative {3}\n\n"
        prompt += template.format(hit['premise'],hit['choice1'],hit['choice2'],int(hit['label'])+1)
    
    prompt += "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative".format(item['premise'], item['choice1'], item['choice2'])
    
    res = generate(prompt, model=model_name)
    
    results_file.write(json.dumps({
        "idx": item['idx'],
        "label": int(res)-1
    })+'\n')

results_file.close()

In [18]:
from tqdm import tqdm
import numpy as np
import json, random

kshot = 3
model_name = "random"

logs = []

results_file = open("../results/random/COPA.jsonl".format(model_name, kshot), "w")

for item in tqdm(test):
    query = "Context: {0} Alternative 1: {1} Alternative 2: {2}".format(item['premise'], item['choice1'], item['choice2'])
    query_embedding = model.encode(query)

    topk = util.dot_score(query_embedding, passages_embeddings)[0].topk(kshot)

    hits = random.sample(train, k=kshot)

    prompt = ""
    for hit in hits:
        template = "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative {3}\n\n"
        prompt += template.format(hit['premise'],hit['choice1'],hit['choice2'],int(hit['label'])+1)
    
    prompt += "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative".format(item['premise'], item['choice1'], item['choice2'])
    
    res = generate(prompt, model=model_name)
    
    results_file.write(json.dumps({
        "idx": item['idx'],
        "label": int(res)-1
    })+'\n')

results_file.close()

100%|██████████| 500/500 [05:42<00:00,  1.46it/s]
